In [1]:
from bs4 import BeautifulSoup
# from selenium import webdriver 
# from selenium.webdriver.common.by import By
import nltk
from nltk.corpus import stopwords
import time
import pandas as pd
import requests

# from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# import re
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import math
from numpy.linalg import norm
import numpy as np
from sklearn.neighbors import NearestNeighbors




## Extracting datas from Imdb

In [2]:
url =  "https://www.imdb.com/chart/top/"
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
headers = {"user-agent" : user_agent}
# driver  = webdriver.Chrome()
source = requests.get(url , headers=headers)
# print(source.status_code)
# driver.find_element(By.ID , "list-view-option-detailed").click()
time.sleep(2)
# print(driver.find_element(By.XaPATH , "html/body/div[1]").find_element(By.CLASS_NAME , "ipc-page-wrapper").get_attribute("innerHTML"))
source = BeautifulSoup(source.content , "html.parser")
# print(source)
# data = []
names_and_links = list()
df = pd.DataFrame(columns=["Name" , "Plot"])
for item in source.find("ul" , class_="ipc-metadata-list").find_all("li"):
    # print(item.find("a")["href"])
    name = item.find("h3" , class_="ipc-title__text").text
    print()
    names_and_links.append((name , 'https://www.imdb.com' + "/".join(item.find("a")["href"].split("/")[0:3] + ["plotsummary/?ref_=tt_stry_pl"])))
# print(names_and_links[0])

# print((list(source.findAll("div" , class_ = "ipc-html-content-inner-div")))[1])
# print(list(map(lambda x: x.text , list(source.findAll("div" , class_ = "ipc-page-section ipc-page-section--base celwidget")))))
    i = 0
for x,y in names_and_links:
    i+=1
    print(i)
    res = requests.get(y , headers=headers)
    # time.sleep(3)

    source = BeautifulSoup(res.content , "html.parser")
    ul = (source.find("ul" , class_ = "ipc-metadata-list"))
                                    #   "ipc-metadata-list ipc-metadata-list--dividers-after sc-d1777989-0 FVBoi ipc-metadata-list--base"
                                    #   "ipc-metadata-list ipc-metadata-list--dividers-after sc-d1777989-0 FVBoi ipc-metadata-list--base"
                                    #   "ipc-metadata-list ipc-metadata-list--dividers-between sc-d1777989-0 FVBoi meta-data-list-full ipc-metadata-list--base"
    # print(ul)
    # print(list(ul.find_all("li"))[1].text)
    plot = ul.find_all("li")[1].text
#     print(y)
#     res = requests.get(y , headers=headers)
#     source = BeautifulSoup(source.content , "html.parser")
#     print(source)
    # plot = item.find("div"  , class_ = "ipc-html-content-inner-div").text
    dict = { 'Name' :x.split(".")[1], 'Plot':plot }
    new_row = pd.DataFrame(dict , index=[x.split(".")[0]])
    df = pd.concat([df , new_row] , ignore_index=True)
df.to_csv("datas.csv")
df.head(5)



























































































































































































































































1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
21

,Name,Plot
0,The Shawshank Redemption,Chronicles the experiences of a formerly succe...
1,The Godfather,"The Godfather ""Don"" Vito Corleone is the head ..."
2,The Dark Knight,Set within a year after the events of Batman B...
3,The Godfather Part II,The continuing saga of the Corleone crime fami...
4,12 Angry Men,"The defense and the prosecution have rested, a..."


## Reading stop words

In [3]:
res = None
with open("stopwords.txt") as f:
    res = f.readlines()
res = list(map(lambda x: x.replace("\n" , "") , res ))
res = list(filter(lambda x : len(x)>1 , res))
stop_words = res
# print(res)
# print(list(filter(lambda x : len(x)<2 , stop_words)) ,)

## Removing Stop words

In [4]:
datas = pd.read_csv("datas.csv")

new_res = list()
for item in datas["Plot"]:
    item = item.replace("." , "").replace("!" , "").replace("?" , "").replace("," , "").replace("'" , '').replace("'" , "")
    x= item.split()
    
    x = list(map(lambda v: v.lower() , x ))
    # print(x)
    # for u in x:
    #     print(u)
    #     if u in res  + ["a"]:
    #         # print("it did")
    #         x.remove(u)
    # print(x)
    x = list(filter(lambda x: x  not in res+["a"] , x))
    # print(x)
    item = " ".join(x)
    new_res.append(item)
    # item = item.lower()
datas["cleaned"] = new_res 
datas.head()
# datas["Plot"]

,Unnamed: 0,Name,Plot,cleaned
0,0,The Shawshank Redemption,Chronicles the experiences of a formerly succe...,chronicles experiences formerly successful ban...
1,1,The Godfather,"The Godfather ""Don"" Vito Corleone is the head ...","godfather ""don"" vito corleone head corleone ma..."
2,2,The Dark Knight,Set within a year after the events of Batman B...,set events batman begins (2005) batman lieuten...
3,3,The Godfather Part II,The continuing saga of the Corleone crime fami...,continuing saga corleone crime family tells st...
4,4,12 Angry Men,"The defense and the prosecution have rested, a...",defense prosecution rested jury filing jury de...


## Counting words in plots for creating dictionary

In [5]:
counter = Counter()
for item in (datas["cleaned"]):
    x= item.split()
    counter.update(x)
# print(len(list(counter)))
# print(list(counter.most_common(1000)))
# print(len(list(counter)))
most_commons = list(map(lambda x : x[0] ,counter.most_common(3000)))
most_commons

['life',
 'time',
 'world',
 '-',
 'family',
 'story',
 'war',
 'love',
 'named',
 'own',
 'wife',
 'police',
 'father',
 'son',
 'soon',
 'lives',
 'day',
 'boy',
 'job',
 'woman',
 'help',
 'film',
 'people',
 'called',
 'including',
 'friend',
 'town',
 'friends',
 'john',
 'despite',
 'begins',
 'home',
 'takes',
 'discovers',
 'sent',
 'set',
 'fight',
 'living',
 'children',
 'able',
 'girl',
 'plan',
 'comes',
 'live',
 'death',
 'city',
 'american',
 'hes',
 'killed',
 'daughter',
 'makes',
 'army',
 'pictures',
 'learns',
 'decides',
 'child',
 'former',
 'eventually',
 'tries',
 'care',
 'crime',
 'mysterious',
 'tells',
 'leads',
 'meet',
 'meets',
 'meanwhile',
 'company',
 'jack',
 'mother',
 'park',
 'house',
 'found',
 'murder',
 'money',
 'kill',
 'battle',
 'escape',
 'power',
 'falls',
 'eve',
 'earth',
 'trying',
 'parents',
 'school',
 'wealthy',
 'following',
 'husband',
 'crew',
 'believes',
 'riganas',
 'powerful',
 'future',
 'ring',
 'lost',
 'little',
 'themse

## Vectorizing Plots and creating tfidf matrice

In [6]:
new_data_frame = pd.DataFrame(columns=  most_commons)
results = []
for item in most_commons:
    c = 0
    for val in datas["cleaned"]:
        if item in val.split():
            c+=1
    # print(c)
    # print(math.log(250/c , 10) +1)
    results.append(math.log(250/c , 10) +1)
# print(max(results))
new_data_frame.loc[0] =  results   
new_data_frame
hint = 0
# print(datas.loc[0])
for i in  range(0,250):
    l = {}
    res = datas["cleaned"].loc[i].split()
    # print(res)
    for j in range(3000):
        item = most_commons[j]
        # print(item)
        # if i == 77 and j == 406:
            # print(res.count(item)/len(res)*new_data_frame.loc[0][j])
        # print(res)
        l[item] = (res.count(item)/len(res))*new_data_frame.loc[0][j]
    # if i == 77 and j == 406:
    
    # print(set(l))
   
    
    new_data_frame = pd.concat([new_data_frame , pd.DataFrame(l , index = [i+1])] )
    # new_data_frame = pd.concat([ pd.DataFrame(l , index = [i+1])  , new_data_frame  ]   )
    
    # new_data_frame = new_data_frame.append(l , ignore_index = True)
# new_data_frame = new_data_frame[:-1]
new_data_frame
    # new_data_frame.loc[i] = 

,life,time,world,-,family,story,war,love,named,own,...,priorities,avenues,fletcher—huggo,1800s,diary,containing,magic,diaries,trick,it—gary
0,1.665546,1.707744,1.744727,1.935542,1.920819,1.853872,1.950782,1.935542,1.950782,1.966576,...,3.39794,3.39794,3.39794,3.39794,3.39794,3.39794,3.39794,3.39794,3.39794,3.39794
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067268,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.000000,0.000000,0.000000,0.000000,0.106712,0.000000,0.036126,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037952,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.000000,0.000000,0.000000,0.000000,0.142283,0.068662,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
247,0.030283,0.000000,0.031722,0.017596,0.087310,0.016853,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
248,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
249,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.042408,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


## Finding Similar Movies to enterd plot

In [7]:

def finding_similiars(new_data_frame,plot):
    plot = plot.replace("." , "").replace("!" , "").replace("?" , "").replace("," , "").replace("'" , '').replace("'" , "")
    plot = plot.split()
    plot = list(map(lambda v: v.lower() , plot ))
    # print(plot)
    # for u in x:
    #     print(u)
    #     if u in res  + ["a"]:
    #         # print("it did")
    #         x.remove(u)
    # print(x)
    plot = list(filter(lambda x: x  not in stop_words+["a"] , plot))
    # print(plot)
    item = " ".join(plot)
    # print(plot)
    l = {}
    for i in range(3000):
        # print(new_data_frame.loc[0][i])
        # print(new_data_frame.loc[0][i])
        # print(plot.count(most_commons[i]))
        # print(((plot.count(most_commons[i])/len(plot))*new_data_frame.loc[0][i]))
        l[most_commons[i]] = (plot.count(most_commons[i])/len(plot))*new_data_frame.loc[0][i]
    # print(l)
    x = pd.DataFrame(l , index = [251])
    mag_x = norm(x.loc[251])
    
    new_data_frame = pd.concat([new_data_frame , x] , ignore_index = True)
    # print(new_data_frame.tail())
    
    # print(list(set(x.loc[])))
    # print(list(x.loc[251]))
        # for x in l.values:
    # print(max(l.values))        
    # print(x)
    # # print(new_data_frame.head())
    # print(np.array(new_data_frame.loc[251]))
    # print("\n")
    # print(np.array(new_data_frame.loc[2]))
        #pd.DataFrame(l , index = [251]))
    # new_data_frame = pd.concat([new_data_frame , pd.DataFrame(l , index = [251])] , ignore_index = True)
    # print(max(new_data_frame.loc[251]))
    # print(new_data_frame.loc[251])
    # print(datas.loc[249])
    final_results = list()
    for i in range(250):
        mag_y = norm(np.array(new_data_frame.loc[i+1]))
        # if i==249 : 
        #     print((np.inner(np.array(new_data_frame.loc[i+1]), np.array(new_data_frame.loc[250]).reshape(3000 , ))/(mag_x*mag_y)))
            # print(new_data_frame.loc[i+1])
            # print(new_data_frame.loc[250])
        # print((new_data_frame.loc[i][0]) , (x.loc[251][0]))
        # print(new_data_frame.loc[i])
        # print((np.inner(new_data_frame.loc[i] , x.loc[251]) , i))
        final_results.append((np.inner(np.array(new_data_frame.loc[i+1]), np.array(new_data_frame.loc[251]).reshape(3000 , ))/(mag_x*mag_y) , i))
        
    seven_best_ones = sorted(final_results , key= lambda x : x[0] , reverse =True )[:5]
    print(seven_best_ones)
    best_film_names = [datas["Name"].loc[y] for x,y in seven_best_ones]
    
    
    # item = item.replace("." , "").replace("!" , "").replace("?" , "").replace("," , "").replace("'" , '').replace("'" , "")
    # x= item.split()
    
    # x = list(map(lambda v: v.lower() , x ))
    
    
    
    # print(x  for x in new_data_frame.loc[173])
    # print(item)
    
    
    # for u in (datas.loc[195]["cleaned"].split()):
    #     # print(u)s
        
    #     if u in item.split() :
    #         print(u)
    # print(item)
    # for key in l.keys():
    #     if l[key]!= 0:
    #         print(key)
    # print(x.loc[251])
    # print(new_data_frame.loc[1])
    # print((list(x.loc[251])))
    # for i in range(1000) :
    #     if list(x.loc[251])[i]!= 0 :
    #         print(i)
    # print("\n")
    # # # print((list(new_data_frame.loc[1])))
    # for i in range(1000) :
    #     if list(new_data_frame.loc[77])[i]!= 0 :
    #         print(most_commons[i])
            
    # for p in item.split():
    #     print(p)
    # print("\n")
    # for item in new_data_frame.loc[57].split(:
    #     print(item)
        # if p in datas.loc[57]["cleaned"].split():
    # print(datas.loc[230]["cleaned"])
    # print(list(new_data_frame.loc[57]) )    
    print(best_film_names)

    # print(new_data_frame)
    # return new_data_frame
# stop_words
finding_similiars(new_data_frame , "The Godfather 'Don' Vito Corleone is the head of the Corleone mafia family in New York. He is at the event of his daughter's wedding. Michael, Vito's youngest son and a decorated WWII Marine is also present at the wedding. Michael seems to be uninterested in being a part of the family business. Vito is a powerful man, and is kind to all those who give him respect but is ruthless against those who do not. But when a powerful and treacherous rival wants to sell drugs and needs the Don's influence for the same, Vito refuses to do it. What follows is a clash between Vito's fading old values and the new ways which may cause Michael to do the thing he was most reluctant in doing and wage a mob war against all the other mafia families which could tear the Corleone family apart.")

[(0.9696506216402684, 1), (0.3801020240877696, 3), (0.10458962953305755, 195), (0.08715093603963064, 246), (0.08441158309112674, 35)]
[' The Godfather', ' The Godfather Part II', ' The Deer Hunter', ' Drishyam', ' Gladiator']


## Downloading stopwords from nltk

In [8]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to C:\Users\ma-
[nltk_data]     na\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\ma-
[nltk_data]     na\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\ma-
[nltk_data]     na\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\ma-
[nltk_data]     na\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Removing stop words using nltk

In [9]:
stop_words = set(stopwords.words('english'))
nltk_res  =list()
for word in datas["Plot"]:
    # print(str(word))
# example_sent = """This is a sample sentence,
#                   showing off the stop words filtration."""
 
    
    word_tokens = word_tokenize(str(word))
    # converts the words in word_tokens to lower case and then checks whether 
    #they are present in stop_words or not
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    #with no lower case conversion
    filtered_sentence = []
    
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    nltk_res.append(" ".join(filtered_sentence))
    # print(word_tokens)
    # print(filtered_sentence)
datas["nltk_cleaned"] = nltk_res
datas["cleaned"]

0      chronicles experiences formerly successful ban...
1      godfather "don" vito corleone head corleone ma...
2      set events batman begins (2005) batman lieuten...
3      continuing saga corleone crime family tells st...
4      defense prosecution rested jury filing jury de...
                             ...                        
245    ellie andrews tied knot society aviator king w...
246    vijay salgaonkar runs cable tv network remote ...
247    shahid khan exiled impersonating legendary sul...
248    lt john dunbar dubbed hero accidentally leads ...
249    aladdin poor street urchin spends time stealin...
Name: cleaned, Length: 250, dtype: object

## Creating the vectorizer and tfidf matrice and creating knn model and fitting it

In [10]:
vectorizer = TfidfVectorizer(min_df=1 , stop_words="english")
tfidf = vectorizer.fit_transform(datas["nltk_cleaned"]).toarray()
# print(tfidf[0])
model_knn = NearestNeighbors(n_neighbors=5,metric = 'cosine', algorithm = 'brute')
model_knn.fit(tfidf)
# tfidf
# print(vectorizer.fit_transform(datas["nltk_cleaned"]))
# for item in list(datas["nltk_cleaned"]):
#     print(vectorizer.fit_transform(item))
# X = vectorizer.fit_transform(list(datas["nltk_cleaned"]))
 # Inspect feature names and TF-IDF values 
# print(vectorizer.get_feature_names_out()) 
# # Convert the variable to an array 
# X.toarray()

NearestNeighbors(algorithm='brute', metric='cosine')

## finding similar films to entered plot using knn model 

In [11]:
# new_data_frame
# def dot(v1, v2):
#     return sum(x*y for , y in zip(v1, v2))
# print(datas.loc[169])
def finding_similiarsـnew(new_data_frame,plot_n , model):
    # input=["input"]
    plot_n= [plot_n]
    plot_n = vectorizer.transform(plot_n).toarray()
    # print(plot_n)
    suggestions = model.kneighbors(plot_n , 5)
    # model.fit(datas["nltk_cleaned"])
    # print(model.kneighbors([plot], 2, return_distance=False))

    # plot = plot.replace("." , "").replace("!" , "").replace("?" , "").replace("," , "").replace("'" , '').replace("'" , "")
    # plot = plot.split()
    # plot = list(map(lambda v: v.lower() , plot ))
    # print(plot)
    # for u in x:
    #     print(u)
    #     if u in res  + ["a"]:
    #         # print("it did")
    #         x.remove(u)
    # print(x)
    # plot = list(filter(lambda x: x  not in stop_words+["a"] , plot))
    # # print(plot)
    # item = " ".join(plot)
    # # print(plot)
    # l = {}
    # for i in range(3000):
    #     # print(new_data_frame.loc[0][i])
    #     # print(new_data_frame.loc[0][i])
    #     # print(plot.count(most_commons[i]))
    #     # print(((plot.count(most_commons[i])/len(plot))*new_data_frame.loc[0][i]))
    #     l[most_commons[i]] = (plot.count(most_commons[i])/len(plot))*new_data_frame.loc[0][i]
    # print(l)
    # x = pd.DataFrame(l , index = [251])
    # mag_x = norm(plot_n)
    for suggestion in suggestions[1][0]:
        # print(suggestion)
        print(datas.loc[suggestion]["Name"])
    # # new_data_frame = pd.concat([new_data_frame , x] , ignore_index = True)
    # # print(new_data_frame.tail())
    
    # # print(list(set(x.loc[])))
    # # print(list(x.loc[251]))
    #     # for x in l.values:
    # # print(max(l.values))        
    # # print(x)
    # # # print(new_data_frame.head())
    # # print(np.array(new_data_frame.loc[251]))
    # # print("\n")
    # # print(np.array(new_data_frame.loc[2]))
    #     #pd.DataFrame(l , index = [251]))
    # # new_data_frame = pd.concat([new_data_frame , pd.DataFrame(l , index = [251])] , ignore_index = True)
    # # print(max(new_data_frame.loc[251]))
    # # print(new_data_frame.loc[251])
    # # print(datas.loc[249])
    # final_results = list()
    # for i in range(250):
    #     mag_y = norm(tfidf[i])
    #     # if i==249 : 
    #     #     print((np.inner(np.array(new_data_frame.loc[i+1]), np.array(new_data_frame.loc[250]).reshape(3000 , ))/(mag_x*mag_y)))
    #         # print(new_data_frame.loc[i+1])
    #         # print(new_data_frame.loc[250])
    #     # print((new_data_frame.loc[i][0]) , (x.loc[251][0]))
    #     # print(new_data_frame.loc[i])
    #     # print((np.inner(new_data_frame.loc[i] , x.loc[251]) , i))
    #     final_results.append((np.inner(tfidf[i] , plot_n)/(mag_y*mag_x) , i))
        
    # seven_best_ones = sorted(final_results , key= lambda x : x[0] , reverse =True )[:7]
    # print(suggestions[1])
    # best_film_names = [datas["Name"].loc[y] for x,y in seven_best_ones]
    
    
    # item = item.replace("." , "").replace("!" , "").replace("?" , "").replace("," , "").replace("'" , '').replace("'" , "")
    # x= item.split()
    
    # x = list(map(lambda v: v.lower() , x ))
    
    
    
    # print(x  for x in new_data_frame.loc[173])
    # print(item)
    
    
    # for u in (datas.loc[195]["cleaned"].split()):
    #     # print(u)s
        
    #     if u in item.split() :
    #         print(u)
    # print(item)
    # for key in l.keys():
    #     if l[key]!= 0:
    #         print(key)
    # print(x.loc[251])
    # print(new_data_frame.loc[1])
    # print((list(x.loc[251])))
    # for i in range(1000) :
    #     if list(x.loc[251])[i]!= 0 :
    #         print(i)
    # print("\n")
    # # # print((list(new_data_frame.loc[1])))
    # for i in range(1000) :
    #     if list(new_data_frame.loc[77])[i]!= 0 :
    #         print(most_commons[i])
            
    # for p in item.split():
    #     print(p)
    # print("\n")
    # for item in new_data_frame.loc[57].split(:
    #     print(item)
        # if p in datas.loc[57]["cleaned"].split():
    # print(datas.loc[230]["cleaned"])
    # print(list(new_data_frame.loc[57]) )    
    # print(best_film_names)

    # print(new_data_frame)
    # return new_data_frame
    
# stop_words
finding_similiarsـnew(new_data_frame , "The Godfather 'Don' Vito Corleone is the head of the Corleone mafia family in New York. He is at the event of his daughter's wedding. Michael, Vito's youngest son and a decorated WWII Marine is also present at the wedding. Michael seems to be uninterested in being a part of the family business. Vito is a powerful man, and is kind to all those who give him respect but is ruthless against those who do not. But when a powerful and treacherous rival wants to sell drugs and needs the Don's influence for the same, Vito refuses to do it. What follows is a clash between Vito's fading old values and the new ways which may cause Michael to do the thing he was most reluctant in doing and wage a mob war against all the other mafia families which could tear the Corleone family apart." ,model_knn)

 The Godfather
 The Godfather Part II
 Trainspotting
 Singin' in the Rain
 Goodfellas
